# Machine Learning: Anime Recommendation

#### Data Collection

In [1]:
import kaggle

In [2]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database --unzip


  0%|          | 0.00/25.0M [00:00<?, ?B/s]
  4%|3         | 1.00M/25.0M [00:00<00:02, 8.98MB/s]
  8%|7         | 2.00M/25.0M [00:00<00:02, 9.33MB/s]
 12%|#1        | 3.00M/25.0M [00:00<00:02, 9.64MB/s]
 20%|#9        | 5.00M/25.0M [00:00<00:02, 10.3MB/s]
 24%|##3       | 6.00M/25.0M [00:00<00:02, 9.88MB/s]
 28%|##7       | 7.00M/25.0M [00:00<00:02, 9.38MB/s]
 36%|###5      | 9.00M/25.0M [00:01<00:02, 8.02MB/s]
 44%|####3     | 11.0M/25.0M [00:01<00:01, 8.56MB/s]
 52%|#####1    | 13.0M/25.0M [00:01<00:01, 9.19MB/s]
 56%|#####5    | 14.0M/25.0M [00:01<00:01, 9.45MB/s]
 60%|#####9    | 15.0M/25.0M [00:01<00:01, 9.61MB/s]
 68%|######7   | 17.0M/25.0M [00:01<00:00, 10.2MB/s]
 72%|#######1  | 18.0M/25.0M [00:01<00:00, 10.2MB/s]
 76%|#######5  | 19.0M/25.0M [00:02<00:00, 9.95MB/s]
 80%|#######9  | 20.0M/25.0M [00:02<00:00, 9.64MB/s]
 88%|########7 | 22.0M/25.0M [00:02<00:00, 10.5MB/s]
 96%|#########5| 24.0M/25.0M [00:02<00:00, 10.9MB/s]
100%|##########| 25.0M/25.0M [00:02<00:00, 10.0MB/s]


##### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

import scipy.stats as st

In [2]:
anime=pd.read_csv("anime.csv", index_col="anime_id")

In [5]:
rating=pd.read_csv("rating.csv")

In [4]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [9]:
anime.isna().sum()

name          0
genre        59
type          0
episodes      0
rating      227
members       0
dtype: int64

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [11]:
anime.shape

(12294, 7)

In [17]:
anime.type.unique()

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA', nan], dtype=object)

In [37]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
# dropping those with NA in "genre", "type", "rating"
anime[anime.loc[:, ["genre", "type", "rating"]].isna().all(axis=1)]
anime.drop(anime[anime.loc[:, ["genre", "type", "rating"]].isna().all(axis=1)].index, axis=0, inplace=True)

In [54]:
anime[anime.episodes=="Unknown"]

,anime_id,name,genre,type,episodes,rating,members
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,Unknown,8.58,504862
252,235,Detective Conan,"Adventure, Comedy, Mystery, Police, Shounen",TV,Unknown,8.25,114702
615,1735,Naruto: Shippuuden,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,Unknown,7.94,533578
991,966,Crayon Shin-chan,"Comedy, Ecchi, Kids, School, Shounen, Slice of...",TV,Unknown,7.73,26267
1021,33157,Tanaka-kun wa Itsumo Kedaruge Specials,"Comedy, School, Slice of Life",Special,Unknown,7.72,5400
...,...,...,...,...,...,...,...
12265,34361,Kyonyuu Reijou MC Gakuen,Hentai,OVA,Unknown,NaN,205
12274,34492,Nuki Doki! Tenshi to Akuma no Sakusei Battle -...,Hentai,OVA,Unknown,NaN,392
12280,34312,Saimin Class,Hentai,OVA,Unknown,NaN,240
12282,34388,Shikkoku no Shaga The Animation,Hentai,OVA,Unknown,NaN,195


In [4]:
anime[anime.genre.fillna("").str.contains("music", case=False)].type.mode()

0    Music
dtype: object

In [55]:
#
anime[anime.type.isna()]

,anime_id,name,genre,type,episodes,rating,members
10898,30484,Steins;Gate 0,"Sci-Fi, Thriller",NaN,Unknown,NaN,60999
10900,34437,Code Geass: Fukkatsu no Lelouch,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",NaN,Unknown,NaN,22748
10906,33352,Violet Evergarden,"Drama, Fantasy",NaN,Unknown,NaN,20564
10907,33248,K: Seven Stories,"Action, Drama, Super Power, Supernatural",NaN,Unknown,NaN,22133
10918,33845,Free! (Shinsaku),"School, Sports",NaN,Unknown,NaN,8666
10949,33475,Busou Shoujo Machiavellianism,"Action, School, Shounen",NaN,Unknown,NaN,1896
10963,31456,Code:Realize: Sousei no Himegimi,"Adventure, Fantasy, Romance, Shoujo",NaN,Unknown,NaN,4017
10988,34280,Gamers!,"Comedy, Romance, School",NaN,Unknown,NaN,1045
10991,32455,Gekidol,Music,NaN,Unknown,NaN,586
10994,31433,Ginga Eiyuu Densetsu (2017),"Action, Drama, Military, Sci-Fi, Space",NaN,Unknown,NaN,3969


In [85]:
!kaggle datasets download -d azathoth42/myanimelist -f AnimeList.csv --unzip


  0%|          | 0.00/3.02M [00:00<?, ?B/s]
 33%|###3      | 1.00M/3.02M [00:00<00:00, 3.55MB/s]
 66%|######6   | 2.00M/3.02M [00:00<00:00, 3.37MB/s]
 99%|#########9| 3.00M/3.02M [00:00<00:00, 3.39MB/s]
100%|##########| 3.02M/3.02M [00:00<00:00, 3.33MB/s]


In [86]:
import zipfile

In [88]:
with zipfile.ZipFile("28524%2F45582%2Fcompressed%2FAnimeList.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("")

In [5]:
animelist=pd.read_csv("AnimeList.csv")

In [6]:
# trying to fill in the missing type information from new animelist, there are still "Unknown"
anime.type.fillna(animelist.set_index("anime_id").type, inplace=True)

In [7]:
anime.type.isna().sum()
anime[anime.type=="Unknown"]

,name,genre,type,episodes,rating,members
anime_id,,,,,,
34437,Code Geass: Fukkatsu no Lelouch,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",Unknown,Unknown,NaN,22748
32455,Gekidol,Music,Unknown,Unknown,NaN,586
28613,Ginga Jinpuu Jinraiger,"Action, Adventure, Mecha",Unknown,Unknown,NaN,627
30448,Mirai Arise,Sci-Fi,Unknown,Unknown,NaN,4044
24023,Project758,Drama,Unknown,Unknown,NaN,455
34348,Rescue Academia,Drama,Unknown,Unknown,NaN,65


In [8]:
# fill in the rest by research
anime_type={34437: "Movie", 32455: "TV", 28613: "TV", 30448: "TV", 24023: "TV", 34348: "TV"}
anime.loc[anime.type=="Unknown", "type"]=anime[anime.type=="Unknown"].index.map(anime_type)

In [15]:
# creating a dummy column for airing using episode as an indicator
anime["airing"]=np.where(anime.episodes=="Unknown", 1, 0)

In [16]:
# creating a dummy column for whether it was NA for rating
anime["rate_na"]=np.where(anime.rating.isna(), 1, 0)

In [188]:
anime[anime.rating.isna()]

,name,genre,type,episodes,rating,members,airing
anime_id,,,,,,,
34502,Inazma Delivery,"Action, Comedy, Sci-Fi",TV,10,NaN,32,0
34309,Nananin no Ayakashi: Chimi Chimi Mouryou!! Gen...,"Comedy, Supernatural",TV,Unknown,NaN,129,1
34096,Gintama (2017),"Action, Comedy, Historical, Parody, Samurai, S...",TV,Unknown,NaN,13383,1
34134,One Punch Man 2,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,Unknown,NaN,90706,1
30484,Steins;Gate 0,"Sci-Fi, Thriller",TV,Unknown,NaN,60999,1
...,...,...,...,...,...,...,...
34492,Nuki Doki! Tenshi to Akuma no Sakusei Battle -...,Hentai,OVA,Unknown,NaN,392,1
34491,Sagurare Otome The Animation,Hentai,OVA,1,NaN,79,0
34312,Saimin Class,Hentai,OVA,Unknown,NaN,240,1


In [167]:
anime.loc[anime.rating.isna(), "airing"]

anime_id
34502    0
34309    1
34096    1
34134    1
30484    1
        ..
34492    1
34491    0
34312    1
34388    1
34399    1
Name: airing, Length: 227, dtype: object

In [139]:
animelist[animelist.anime_id.isin(missing_type)][["anime_id", "type"]]

,anime_id,type
55,33185,Movie
639,33248,Movie
1893,33478,TV
2246,34262,TV
2415,34280,TV
3235,28613,Unknown
3414,31456,TV
4681,24023,Unknown
5029,32455,Unknown
5192,34382,TV


In [11]:
anime.describe(include="all")

,name,genre,type,episodes,rating,members,airing
count,12291,12232,12291,12291,12064.000000,1.229100e+04,12291
unique,12289,3264,6,187,NaN,NaN,2
top,Saru Kani Gassen,Hentai,TV,1,NaN,NaN,0
freq,2,823,3805,5677,NaN,NaN,11954
mean,NaN,NaN,NaN,NaN,6.473902,1.807560e+04,NaN
std,NaN,NaN,NaN,NaN,1.026746,5.482669e+04,NaN
min,NaN,NaN,NaN,NaN,1.670000,5.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,5.880000,2.255000e+02,NaN
50%,NaN,NaN,NaN,NaN,6.570000,1.551000e+03,NaN
75%,NaN,NaN,NaN,NaN,7.180000,9.443000e+03,NaN
